# Download data

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c ifood-2019-fgvc6

Saving kaggle.json to kaggle.json
kaggle.json
  0% 0.00/554k [00:00<?, ?B/s]
100% 554k/554k [00:00<00:00, 75.7MB/s]
  0% 0.00/554k [00:00<?, ?B/s]
100% 554k/554k [00:00<00:00, 77.2MB/s]
 99% 2.10G/2.12G [01:08<00:00, 57.0MB/s]
100% 2.12G/2.12G [01:08<00:00, 33.2MB/s]
  0% 0.00/3.63k [00:00<?, ?B/s]
100% 3.63k/3.63k [00:00<00:00, 3.69MB/s]
 99% 521M/526M [00:13<00:00, 48.8MB/s]
100% 526M/526M [00:13<00:00, 40.9MB/s]
  0% 0.00/217k [00:00<?, ?B/s]
100% 217k/217k [00:00<00:00, 55.2MB/s]
 99% 218M/221M [00:04<00:00, 64.7MB/s]
100% 221M/221M [00:04<00:00, 51.7MB/s]
  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 85.9MB/s]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir data
!unzip train_set.zip -d data
!unzip val_set.zip -d data
!unzip test_set.zip -d data
!unzip train_labels.csv.zip -d data

Streaming output truncated to the last 5000 lines.
  inflating: data/test_set/test_009481.jpg  
  inflating: data/test_set/test_013960.jpg  
  inflating: data/test_set/test_002362.jpg  
  inflating: data/test_set/test_005803.jpg  
  inflating: data/test_set/test_015388.jpg  
  inflating: data/test_set/test_002764.jpg  
  inflating: data/test_set/test_021964.jpg  
  inflating: data/test_set/test_013738.jpg  
  inflating: data/test_set/test_016448.jpg  
  inflating: data/test_set/test_022310.jpg  
  inflating: data/test_set/test_002203.jpg  
  inflating: data/test_set/test_003963.jpg  
  inflating: data/test_set/test_022431.jpg  
  inflating: data/test_set/test_003303.jpg  
  inflating: data/test_set/test_006227.jpg  
  inflating: data/test_set/test_016860.jpg  
  inflating: data/test_set/test_009457.jpg  
  inflating: data/test_set/test_020554.jpg  
  inflating: data/test_set/test_003655.jpg  
  inflating: data/test_set/test_006939.jpg  
  inflating: data/test_set/test_003241.jpg  
  in

#VGG Model

## import libaries

In [0]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import random
random.seed(1)

## Data preparation

In [0]:
# create class to label
def createclass2label(path):
    table = pd.read_table(path, header=None, sep=' ')
    table.columns = ['label', 'class']
    class_dic = {}
    n = len(table)
    for i in range(n):
        class_dic[table.iloc[i]['class']] = table.iloc[i]['label']
    return class_dic

In [0]:
class_list_path = os.path.join("class_list.txt")
class FoodDataset(Dataset):
    def __init__(self, data_dir, data_df, transform=None, class_list_dir=class_list_path):
        self.food_label = createclass2label(class_list_dir)
        self.data_dir = data_dir
        self.data_info = data_df
        self.transform = transform

    def __getitem__(self, index):
        path_img = os.path.join(self.data_dir, self.data_info.iloc[index]['img_name'])
        label = self.data_info.iloc[index]['label']
        img = Image.open(path_img).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.data_info)

In [0]:
train_label_dir = os.path.join("data", "train_labels.csv")
valid_label_dir = os.path.join("val_labels.csv")
test_label_dir = os.path.join("sample_submission.csv")

In [0]:
train_df = pd.read_csv(train_label_dir)
valid_df = pd.read_csv(valid_label_dir)

In [0]:
MAX_EPOCH = 10
BATCH_SIZE = 16
LR = 0.001
log_interval = 10
#check points
val_interval = 1
N_CLASSES = 251
TRAIN_SAMPLE = 50000
VALID_SAMPLE = 5000
checkpoint_interval = 1

In [0]:
train_data_df = train_df.sample(TRAIN_SAMPLE)
valid_data_df = valid_df.sample(VALID_SAMPLE)
test_data_df = pd.read_csv(test_label_dir)

In [0]:
print(f'Found {train_data_df.shape[0]} train images in {len(train_data_df.label.unique())} classes')
print(f'Found {valid_data_df.shape[0]} valid images in {len(valid_data_df.label.unique())} classes')
print(f'Found {test_data_df.shape[0]} test images in {len(test_data_df.label.unique())} classes')

Found 50000 train images in 251 classes
Found 5000 valid images in 250 classes
Found 28377 test images in 1 classes


In [0]:
data_dir = os.path.join("data")
train_dir = os.path.join(data_dir, "train_set")
valid_dir = os.path.join(data_dir, "val_set")
test_dir = os.path.join(data_dir, "test_set")

In [0]:
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

#data augmentation
train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

valid_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

train_data = FoodDataset(data_dir=train_dir, data_df=train_data_df, transform=train_transform)
valid_data = FoodDataset(data_dir=valid_dir, data_df=valid_data_df, transform=valid_transform)
test_data = FoodDataset(data_dir=test_dir, data_df=test_data_df, transform=test_transform)

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE)

## Select model

In [0]:
vgg_model = models.vgg16(pretrained=True)
vgg_model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:
# transfer learning, change classifier
for parameter in vgg_model.parameters():
    parameter.requires_grad = False

classifier = nn.Sequential(OrderedDict({
    'fc1': nn.Linear(25088, 5000),
    'relu': nn.ReLU(),
    'drop': nn.Dropout(p=0.5),
    'fc2': nn.Linear(5000, N_CLASSES),
    'output': nn.LogSoftmax(dim=1)
}))

vgg_model.classifier = classifier

In [0]:
vgg_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Loss Function

In [0]:
# Loss Function
# criterion = nn.NLLLoss()
#cross entropty
criterion = nn.MultiMarginLoss()

## Choose optimizer

In [0]:
# optimizer
optimizer = optim.Adam(vgg_model.parameters(), lr=LR)
#change lr
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)

# Training

In [0]:
def train_classifier(model, print_every):
    train_curve = list()
    valid_curve = list()
    model.to('cuda')
    start_epoch = -1
    for e in range(start_epoch + 1, MAX_EPOCH):
        loss_mean = 0.
        correct = 0.
        total = 0.

        model.train()
        for i, data in enumerate(train_loader):

            # forward
            images, labels = data
            images, labels = images.to('cuda'), labels.to('cuda')
            output = model.forward(images)

            # backward
            optimizer.zero_grad()
            loss = criterion(output, labels)
            loss.backward()

            # update weights
            optimizer.step()

            # calculate loss and training infomation
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).squeeze().sum().cpu().numpy()

            # print
            loss_mean += loss.item()
            train_curve.append(loss.item())
            if (i + 1) % print_every == 0:
                loss_mean = loss_mean / print_every
                print("Training:Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                    e, MAX_EPOCH, i + 1, len(train_loader), loss_mean, correct / total))
                loss_mean = 0.

        scheduler.step()

        if (e + 1) % checkpoint_interval == 0:

            checkpoint = {
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "epoch": e
            }
            path_checkpoint = f'./drive/My Drive/COMP540/checkpoints/vgg16/checkpoint_{e}_epoch.pkl'
            torch.save(checkpoint, path_checkpoint)

        # validation
        if (e + 1) % val_interval == 0:

            correct_val = 0.
            total_val = 0.
            loss_val = 0.
            model.eval()
            with torch.no_grad():
                for j, data in enumerate(valid_loader):
                    images, labels = data
                    images, labels = images.to('cuda'), labels.to('cuda')

                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    _, predicted = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).squeeze().sum().cpu().numpy()

                    loss_val += loss.item()

                loss_val_mean = loss_val / len(valid_loader)
                valid_curve.append(loss.item())
                print("Valid:\t Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                    e, MAX_EPOCH, j + 1, len(valid_loader), loss_val_mean, correct_val / total_val))
            model.train()

In [0]:
train_classifier(vgg_model, 50)

Training:Epoch[000/010] Iteration[050/3125] Loss: 4.9708 Acc:1.38%
Training:Epoch[000/010] Iteration[100/3125] Loss: 1.0273 Acc:1.38%
Training:Epoch[000/010] Iteration[150/3125] Loss: 1.0021 Acc:1.21%
Training:Epoch[000/010] Iteration[200/3125] Loss: 1.0279 Acc:1.16%
Training:Epoch[000/010] Iteration[250/3125] Loss: 0.9838 Acc:1.07%
Training:Epoch[000/010] Iteration[300/3125] Loss: 1.0154 Acc:1.04%
Training:Epoch[000/010] Iteration[350/3125] Loss: 0.9895 Acc:1.11%
Training:Epoch[000/010] Iteration[400/3125] Loss: 1.0105 Acc:1.12%
Training:Epoch[000/010] Iteration[450/3125] Loss: 1.0114 Acc:1.06%
Training:Epoch[000/010] Iteration[500/3125] Loss: 0.9836 Acc:0.99%
Training:Epoch[000/010] Iteration[550/3125] Loss: 0.9491 Acc:0.97%
Training:Epoch[000/010] Iteration[600/3125] Loss: 1.0341 Acc:1.01%
Training:Epoch[000/010] Iteration[650/3125] Loss: 0.9795 Acc:1.00%
Training:Epoch[000/010] Iteration[700/3125] Loss: 0.9942 Acc:1.05%
Training:Epoch[000/010] Iteration[750/3125] Loss: 0.9816 Acc:1

In [0]:
def save_model(model, name, save_state_dic=False):
    model_path = f'./drive/My Drive/COMP540/modelsaves/vgg16/{name}.pkl'
    if save_state_dic:
        path_state_dict = f'./drive/My Drive/COMP540/modelsaves/vgg16/{name}_state_dict.pkl'
        model_state_dict = model.state_dict()
        torch.save(model_state_dict, path_state_dict)
    torch.save(model, model_path)

In [0]:
def load_model(model_path, state_dic_path=None):

    model_load = torch.load(model_path)
    if state_dic_path is not None:
        state_dict_load = torch.load(state_dic_path)
        return model_load, state_dict_load
    return model_load

## Resume training

In [0]:
def train_classifier_resume(model, optimizer, path_checkpoint):
    checkpoint = torch.load(path_checkpoint)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    train_curve = list()
    valid_curve = list()

    print_every = 40

    vgg_model.to('cuda')
    for e in range(start_epoch + 1, MAX_EPOCH):

        loss_mean = 0.
        correct = 0.
        total = 0.

        model.train()
        for i, data in enumerate(train_loader):

            # forward
            images, labels = data
            images, labels = images.to('cuda'), labels.to('cuda')
            output = model.forward(images)

            # backward
            optimizer.zero_grad()
            loss = criterion(output, labels)
            loss.backward()

            # update weights
            optimizer.step()

            # calculate loss and training infomation
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).squeeze().sum().numpy()

            # print
            loss_mean += loss.item()
            train_curve.append(loss.item())
            if (i + 1) % print_every == 0:
                loss_mean = loss_mean / print_every
                print("Training:Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                    e, MAX_EPOCH, i + 1, len(train_loader), loss_mean, correct / total))
                loss_mean = 0.

        scheduler.step()

        if (e + 1) % checkpoint_interval == 0:
            checkpoint = {
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "epoch": e
            }
            path_checkpoint = f'./drive/My Drive/COMP540/checkpoints/vgg16/checkpoint_{e}_epoch.pkl'
            torch.save(checkpoint, path_checkpoint)

        # validation
        if (e + 1) % val_interval == 0:

            correct_val = 0.
            total_val = 0.
            loss_val = 0.
            model.eval()
            with torch.no_grad():
                for j, data in enumerate(valid_loader):
                    images, labels = data
                    images, labels = images.to('cuda'), labels.to('cuda')

                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    _, predicted = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).squeeze().sum().numpy()

                    loss_val += loss.item()

                loss_val_mean = loss_val / len(valid_loader)
                valid_curve.append(loss.item())
                print("Valid:\t Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                    e, MAX_EPOCH, j + 1, len(valid_loader), loss_val_mean, correct_val / total_val))
            model.train()

In [0]:
path_checkpoint = None
train_classifier_resume(vgg_model, optimizer, path_checkpoint)

## Save model

In [0]:
name = 'vgg17'
save_model(vgg_model, name, save_state_dic=True)

In [0]:
torch.save(vgg_model, )

## Inference

In [0]:
##TODO##